In [6]:
import numpy as np
from collections import defaultdict
import random

# Step 1: Initialize corpus
corpus = [
    "the quick brown fox jumps over the lazy dog",
    "i love natural language processing",
    "word2vec is a powerful model for learning word embeddings"
]
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

# Parameters
window_size = 2
embedding_dim = 10
learning_rate = 0.01
epochs = 10000

# Step 2: Build Vocabulary
vocab = set(word for sentence in tokenized_corpus for word in sentence)
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)

# Step 3: Generate Training Data (CBOW)
def generate_training_data(tokenized_corpus, window_size):
    training_data = []
    for sentence in tokenized_corpus:
        for idx, target_word in enumerate(sentence):
            context = []
            for neighbor in range(-window_size, window_size + 1):
                context_pos = idx + neighbor
                if neighbor != 0 and 0 <= context_pos < len(sentence):
                    context.append(sentence[context_pos])
            if context:
                training_data.append((context, target_word))
    return training_data

training_data = generate_training_data(tokenized_corpus, window_size)

# Step 4: Initialize weights
W1 = np.random.rand(vocab_size, embedding_dim)
W2 = np.random.rand(embedding_dim, vocab_size)

# Step 5: Softmax function
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# Step 6 & 7: Forward and Backward propagation
def forward_backward(context_words, target_word):
    global W1, W2
    # One-hot encoding for context
    x = np.zeros(vocab_size)
    for word in context_words:
        x[word2idx[word]] += 1
    x /= len(context_words)

    # Forward pass
    h = np.dot(W1.T, x)  # hidden layer
    u = np.dot(W2.T, h)  # output layer
    y_pred = softmax(u)

    # True target
    y_true = np.zeros(vocab_size)
    y_true[word2idx[target_word]] = 1

    # Error
    error = y_pred - y_true

    # Backward pass
    dW2 = np.outer(h, error)
    dW1 = np.outer(x, np.dot(W2, error))

    # Update weights
   
    W1 -= learning_rate * dW1
    W2 -= learning_rate * dW2

    # Loss (cross-entropy)
    loss = -np.log(y_pred[word2idx[target_word]] + 1e-9)
    return loss

# Step 8: Training
for epoch in range(epochs):
    total_loss = 0
    for context_words, target_word in training_data:
        loss = forward_backward(context_words, target_word)
        total_loss += loss
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")

# Step 9: Get word vectors
def get_word_vector(word):
    if word in word2idx:
        return W1[word2idx[word]]
    else:
        return None

# Example
print("\nWord vector for 'language':")
print(get_word_vector('language'))


Epoch 0, Loss: 75.0772
Epoch 1000, Loss: 2.3654
Epoch 2000, Loss: 0.6350
Epoch 3000, Loss: 0.3295
Epoch 4000, Loss: 0.2147
Epoch 5000, Loss: 0.1564
Epoch 6000, Loss: 0.1218
Epoch 7000, Loss: 0.0990
Epoch 8000, Loss: 0.0830
Epoch 9000, Loss: 0.0712

Word vector for 'language':
[ 1.28902794  0.30396502  1.19111738  2.02477027  1.69908188 -0.38941468
 -0.96064573  0.25709336  2.30716776 -1.84344052]
